In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim

In [2]:
from gensim import corpora,models

In [3]:
stop = stopwords.words('english')

In [4]:
a1 = pd.read_csv('content_reading1.csv',index_col=0)
a2 = pd.read_csv('content_reading2.csv',index_col=0)
a3 = pd.read_csv('content_reading3.csv',index_col=0)
a4 = pd.read_csv('content_reading4.csv',index_col=0)

In [5]:
documents = pd.concat([a1,a2,a3,a4],ignore_index=True)

In [6]:
documents = documents.drop_duplicates()

In [7]:
documents

,content
0,There are some lovely people here who'll help ...
1,"""Aha!!"" Answers - little chat, just post your ..."
2,Most people seem to get started with the Yijin...
3,VisionThe I Ching Community is a warm and open...
4,This forum is for people to share their I Chin...
...,...
16000,A blurred relationship in the middle of an unc...
16001,"Second time my commentaries had readed as ""my ..."
16002,"I asked ""what am I made of? do I have it with..."
16003,"Hello All. I just joined your little group, an..."


In [33]:
documents['content'][5]

"Further to this thread.... https://www.onlineclarity.co.uk/friends/index.php?threads/what-is-the-situation-60-8.31679/The owner/boss has been a bit flakey (or something).I cold canvassed this job (via email) a couple of months ago and she said there's no work try again closer to Easter but a couple of weeks ago I asked again. She emailed me suggesting we meet but then she said she couldn't meet at the times I suggested but why don't I just go into the (childcare) centre and see if I like it and if I like it, let her know and we'll set up a meeting from there the following week.I did all she asked of me but she didn't respond within a week (before that she'd been a very effective communicator) so I emailed her again thanking her for her time but I'm assuming she's not interested.48 hours later she emailed me apologising for her 'tardiness' made her excuses and suggested we meet at a certain time which she ended up changing and again I fitted in around her.She never promised me more tha

In [8]:
documents['content'][16004]

'I\'m on a two-month stay living with my parents as I work from home at my office job. This is a temporary situation; i haven\'t lived with them (for more than a two-week stay) in more than 16 years.My mother has always had controlling tendencies but my stay has been OK so far until tonight. She sat me down to have a conversation about some intimate details of my life that I do not discuss with her and that IMHO are none of her business -- not as a mother. There are certain things I just do not and cannot share with her and these are top of the list. Partly bc growing up these things were never discussed. In actuality there is much about my life she doesn\'t know and that is fine by me bc she knows more than most parents do.However in our "discussion"(in which she attacked me and accused me of not wanting to discuss it) she blew up and brought up issues from 16 years ago. I had been dreading this before I came for the visit bc i never know what lies temporarily dormant beneath the surf

In [11]:
a = documents.duplicated()

In [15]:
documents.index[a]

Int64Index([2535, 4005, 5233, 12356, 14458, 14799, 15109, 15934], dtype='int64')

In [27]:
documents.iloc[4005,:]

content    It's Monday morning and I have been seized wit...
Name: 4005, dtype: object

In [17]:
documents.iloc[2535,:]

content    Hello everyone,There is this female dj I have ...
Name: 2535, dtype: object

In [8]:
corpus=[]
for doc in documents.content:
    text=re.sub('[^a-zA-z]',' ', doc)
    text=text.lower()
    text=text.split()
    text=[PorterStemmer().stem(word) for word in text if not word in stop]
    text=' '.join(text)
    corpus.append(text)

In [9]:
documents['clean_content'] = corpus

In [10]:
documents.head()

,content,clean_content
0,There are some lovely people here who'll help ...,love peopl help understand read make easier he...
1,"""Aha!!"" Answers - little chat, just post your ...",aha answer littl chat post favourit encourag r...
2,Most people seem to get started with the Yijin...,peopl seem get start yije ask relationship goo...
3,VisionThe I Ching Community is a warm and open...,visionth ching commun warm open place free wid...
4,This forum is for people to share their I Chin...,forum peopl share ching read exchang insight m...


In [11]:
processed_docs = documents.clean_content.str.split()

In [12]:
processed_docs

0        [love, peopl, help, understand, read, make, ea...
1        [aha, answer, littl, chat, post, favourit, enc...
2        [peopl, seem, get, start, yije, ask, relations...
3        [visionth, ching, commun, warm, open, place, f...
4        [forum, peopl, share, ching, read, exchang, in...
                               ...                        
16000    [blur, relationship, middl, unclear, break, as...
16001    [second, time, commentari, read, point, view, ...
16002    [ask, made, within, get, truli, want, life, fa...
16003    [hello, join, littl, group, want, introduc, wo...
16004    [two, month, stay, live, parent, work, home, o...
Name: clean_content, Length: 16005, dtype: object

In [13]:
dictionary=gensim.corpora.Dictionary(processed_docs)

In [14]:
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>20:
        break

0 actual
1 ad
2 address
3 answer
4 archiv
5 ask
6 aug
7 b
8 background
9 besid
10 brows
11 click
12 come
13 contact
14 context
15 easier
16 edit
17 eg
18 emphasi
19 enough
20 everyon


In [15]:
dictionary.filter_extremes(no_below=10,no_above=0.3,keep_n=1000)

In [16]:
len(dictionary)

1000

In [17]:
dictionary[999]

'apr'

In [18]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]

In [19]:
len(bow_corpus)

16005

In [21]:
bow_corpus_100=bow_corpus[100]
for i in range(len(bow_corpus_100)):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus_100[i][0], 
                                                     dictionary[bow_corpus_100[i][0]], 
                                                     bow_corpus_100[i][1]))

Word 11 ("everyon") appears 1 time.
Word 17 ("hexagram") appears 3 time.
Word 26 ("need") appears 1 time.
Word 28 ("peopl") appears 1 time.
Word 36 ("result") appears 2 time.
Word 38 ("say") appears 1 time.
Word 49 ("understand") appears 1 time.
Word 59 ("ching") appears 1 time.
Word 77 ("give") appears 1 time.
Word 92 ("mean") appears 1 time.
Word 99 ("often") appears 1 time.
Word 104 ("place") appears 1 time.
Word 109 ("relationship") appears 1 time.
Word 121 ("tell") appears 1 time.
Word 135 ("alreadi") appears 1 time.
Word 166 ("could") appears 2 time.
Word 193 ("futur") appears 1 time.
Word 205 ("lead") appears 1 time.
Word 213 ("may") appears 1 time.
Word 229 ("possibl") appears 1 time.
Word 233 ("provid") appears 1 time.
Word 251 ("stop") appears 1 time.
Word 260 ("tri") appears 1 time.
Word 267 ("without") appears 1 time.
Word 268 ("work") appears 1 time.
Word 273 ("action") appears 1 time.
Word 302 ("given") appears 1 time.
Word 309 ("job") appears 1 time.
Word 312 ("left") ap

In [22]:
tfidf=models.TfidfModel(bow_corpus)

In [23]:
corpus_tfidf=tfidf[bow_corpus]

In [24]:
corpus_tfidf[100]

[(11, 0.08297869630694811),
 (17, 0.1370132360948113),
 (26, 0.04835157266884867),
 (28, 0.06510638239107026),
 (36, 0.1698853831903319),
 (38, 0.04562325531109188),
 (49, 0.06501127121248618),
 (59, 0.06017703598141311),
 (77, 0.06681950211381063),
 (92, 0.04659187116447117),
 (99, 0.10991160795843409),
 (104, 0.07844841578266366),
 (109, 0.05099517351094465),
 (121, 0.06840810328339125),
 (135, 0.09728210466304113),
 (166, 0.1014568356330565),
 (193, 0.09173322481123904),
 (205, 0.10153984540353521),
 (213, 0.0752735810421925),
 (229, 0.08061184875054277),
 (233, 0.13565395374940917),
 (251, 0.09479976141285056),
 (260, 0.05538573278389249),
 (267, 0.08534379197338668),
 (268, 0.048480582348096886),
 (273, 0.09821618030549283),
 (302, 0.11033343152383869),
 (309, 0.07809678413507908),
 (312, 0.10062584476913333),
 (315, 0.073142445269371),
 (342, 0.13509115502249736),
 (357, 0.09270759965738716),
 (393, 0.133900818223394),
 (419, 0.12162965584201241),
 (421, 0.13408143195590816),
 (4

### Model

In [25]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=4, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [26]:
lda_model_tfidf.log_perplexity(bow_corpus)

-6.52392729056418

In [27]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.007*"relationship" + 0.007*"answer" + 0.006*"hex" + 0.006*"yi" + 0.005*"hexagram" + 0.005*"x" + 0.005*"seem" + 0.005*"interpret" + 0.004*"receiv" + 0.004*"situat"


Topic: 1 Word: 0.007*"hexagram" + 0.006*"relationship" + 0.005*"pleas" + 0.005*"ching" + 0.005*"mean" + 0.005*"year" + 0.004*"answer" + 0.004*"seem" + 0.004*"want" + 0.004*"new"


Topic: 2 Word: 0.007*"job" + 0.005*"work" + 0.005*"hexagram" + 0.004*"hex" + 0.004*"want" + 0.004*"good" + 0.004*"say" + 0.004*"realli" + 0.004*"need" + 0.004*"back"


Topic: 3 Word: 0.007*"hexagram" + 0.006*"edit" + 0.005*"hex" + 0.005*"last" + 0.004*"move" + 0.004*"relationship" + 0.004*"mean" + 0.004*"good" + 0.004*"say" + 0.004*"yi"




In [28]:
a = lda_model_tfidf[corpus_tfidf[100]]
a

[(0, 0.037964877), (1, 0.03749353), (2, 0.038273867), (3, 0.8862677)]

In [29]:
topic=[]
for i in range(len(corpus_tfidf)):
    a = lda_model_tfidf[corpus_tfidf[i]]
    a.sort(key=lambda x: x[1],reverse=True)
    topic.append([l[0] for l in a if l[1]>0.5])

In [30]:
T = []
for t in topic:
    T.extend(t)

In [31]:
for i in set(T):
    print(f'Topic {i} have {T.count(i)} times')

Topic 0 have 4349 times
Topic 1 have 3606 times
Topic 2 have 4983 times
Topic 3 have 2850 times


In [ ]:
hexagram,edit,last,move,relationship,mean,good